In [1]:
# import pandas and set colwidth to view
import pandas as pd
pd.options.display.max_colwidth = 1000

In [2]:
# read data
df_cases = pd.read_pickle('../data/df_cases_200906.gzip')
df_label = pd.read_pickle('../data/df_label_200906.gzip')

In [3]:
df_cases.head()

,CaseId,FileName,Language,StartDate,DocumentType,IsExecuted,OcrText,QualityScore
0,003061226227,003061226227_12045631_Order form_978-1-67767-388-9.pdf,EN,20191104,Order form,False,Pressure style response character.,0.649292
1,003061226383,003061226383_63912371_Contract Info Pack_978-0-394-16412-0.pdf,EN,20190805,Contract Info Pack,False,Soon especially boy thousand traditional.,0.873038
2,003061226383,003061226383_41775424_other documents_978-1-388-73116-8.pdf,EN,20190805,other documents,True,Tax south say strategy hard between late.,0.991280
3,003061227777,003061227777_96463321_other documents_978-1-960601-99-5.pdf,EN,20191231,other documents,False,Yet arm six design.,0.607833
4,003061227781,003061227781_62201704_other documents_978-1-04-652792-8.pdf,EN,20191231,other documents,False,Plan camera actually run push rest.,0.909359


In [4]:
df_label.head()

,CaseId,label_1,label_2
0,003061226227,False,False
1,003061226383,True,False
2,003061227777,False,False
3,003061227781,False,False
4,003061227680,True,True


In [5]:
# list Invalid and Valid, Concat valid OcrText
df = df_cases.groupby('CaseId').apply(lambda g: pd.Series({  
                                              'InvalidFileNames': [', '.join(g['FileName'][(g.IsExecuted == False)|(g.QualityScore<0.81)])],
                                              'ValidFileNames': [', '.join(g['FileName'][~((g.IsExecuted == False)|(g.QualityScore<0.81))])],
                                              'OcrText': ' '.join(g['OcrText'][~((g.IsExecuted == False)|(g.QualityScore<0.81))])
                                             })).reset_index()

In [6]:
# merge label to df
df = df.merge(df_label, how='left', on='CaseId')
df.head()

,CaseId,InvalidFileNames,ValidFileNames,OcrText,label_1,label_2
0,003061189006,[],[003061189006_69176036_Order form_978-1-62414-909-2.pdf],As difficult behavior her myself help.,True,False
1,003061189067,[],[003061189067_26173467_Order form_978-1-904782-31-5.pdf],Present can phone form.,True,False
2,003061189156,[],[003061189156_47966765_Contract Documents_978-0-317-34803-3.pdf],Section science difference success wish it wide.,False,False
3,003061189229,[003061189229_69115288_Master contract_978-0-11-692367-7.pdf],[003061189229_21094545_other documents_978-1-81560-843-8.pdf],Accept sell leader herself if.,False,False
4,003061189242,[003061189242_91658807_other documents_978-0-7143-5356-2.pdf],[],,False,False


In [7]:
# export to gzip
df.to_pickle("df_final.gzip")